## 📦 1. Imports et Configuration

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from datetime import datetime
import json

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

from sklearn.metrics import (
    precision_score, recall_score, f1_score,
    hamming_loss, accuracy_score, classification_report,
    roc_auc_score, roc_curve, auc
)

# Configuration
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

# Seed pour reproductibilité
np.random.seed(42)
tf.random.set_seed(42)

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU disponible: {len(tf.config.list_physical_devices('GPU')) > 0}")
print("\n✅ Imports réussis!")

## 📂 2. Chargement des Données Préparées

In [ ]:
# Monter Google Drive
from google.colab import drive
drive.mount('/content/drive')

print("Chargement des données depuis Google Drive...\n")

# Définir le chemin vers les données dans Drive
DATA_PATH = '/content/drive/MyDrive/emotion_detection_project/processed'

# Charger les séquences
X_train = np.load(f'{DATA_PATH}/X_train.npy')
X_val = np.load(f'{DATA_PATH}/X_val.npy')
X_test = np.load(f'{DATA_PATH}/X_test.npy')
print("✅ Séquences chargées")

# Charger les labels
y_train = np.load(f'{DATA_PATH}/y_train.npy')
y_val = np.load(f'{DATA_PATH}/y_val.npy')
y_test = np.load(f'{DATA_PATH}/y_test.npy')
print("✅ Labels chargés")

# Charger les métadonnées
with open(f'{DATA_PATH}/metadata.pkl', 'rb') as f:
    metadata = pickle.load(f)
print("✅ Métadonnées chargées")

# Extraire les paramètres
emotion_labels = metadata['emotion_labels']
NUM_CLASSES = metadata['num_classes']
MAX_VOCAB_SIZE = metadata['max_vocab_size']
MAX_SEQUENCE_LENGTH = metadata['max_sequence_length']

print(f"\n📊 Configuration:")
print(f"  Train samples: {len(X_train):,}")
print(f"  Val samples:   {len(X_val):,}")
print(f"  Test samples:  {len(X_test):,}")
print(f"  Num classes:   {NUM_CLASSES}")
print(f"  Vocab size:    {MAX_VOCAB_SIZE:,}")
print(f"  Seq length:    {MAX_SEQUENCE_LENGTH}")

## 🏗️ 3. Construction du Modèle LSTM

In [ ]:
def build_lstm_model(vocab_size, embedding_dim=128, lstm_units=64, 
                     num_classes=28, max_length=128, dropout_rate=0.5):
    """
    Construire un modèle LSTM simple pour classification multi-label
    
    Args:
        vocab_size: Taille du vocabulaire
        embedding_dim: Dimension des embeddings
        lstm_units: Nombre d'unités LSTM
        num_classes: Nombre de classes (émotions)
        max_length: Longueur maximale des séquences
        dropout_rate: Taux de dropout
    
    Returns:
        Modèle Keras compilé
    """
    model = keras.Sequential([
        # Embedding layer
        layers.Embedding(
            input_dim=vocab_size,
            output_dim=embedding_dim,
            input_length=max_length,
            mask_zero=True,
            name='embedding'
        ),
        
        # LSTM layer
        layers.LSTM(lstm_units, name='lstm'),
        
        # Dropout pour régularisation
        layers.Dropout(dropout_rate, name='dropout'),
        
        # Output layer (sigmoid pour multi-label)
        layers.Dense(num_classes, activation='sigmoid', name='output')
    ], name='SimpleLSTM')
    
    # Compiler le modèle
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=0.001),
        loss='binary_crossentropy',
        metrics=[
            'accuracy',
            keras.metrics.Precision(name='precision'),
            keras.metrics.Recall(name='recall')
        ]
    )
    
    return model

# Construire le modèle
print("Construction du modèle LSTM...\n")
lstm_model = build_lstm_model(
    vocab_size=MAX_VOCAB_SIZE,
    embedding_dim=128,
    lstm_units=64,
    num_classes=NUM_CLASSES,
    max_length=MAX_SEQUENCE_LENGTH,
    dropout_rate=0.5
)

# Afficher l'architecture
lstm_model.summary()

print(f"\n✅ Modèle LSTM construit!")
print(f"  Paramètres totaux: {lstm_model.count_params():,}")

## 🎓 4. Entraînement du Modèle

In [ ]:
# Callbacks
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True,
    verbose=1
)

model_checkpoint = ModelCheckpoint(
    'models/lstm/best_model.h5',
    monitor='val_loss',
    save_best_only=True,
    verbose=1
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=3,
    min_lr=0.00001,
    verbose=1
)

print("\n" + "="*60)
print("🚀 DÉBUT DE L'ENTRAÎNEMENT")
print("="*60)

start_time = datetime.now()
print(f"Heure de début: {start_time.strftime('%H:%M:%S')}\n")

# Entraînement
history = lstm_model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=20,
    batch_size=32,
    callbacks=[early_stopping, model_checkpoint, reduce_lr],
    verbose=1
)

end_time = datetime.now()
training_time = (end_time - start_time).total_seconds()

print("\n" + "="*60)
print("✅ ENTRAÎNEMENT TERMINÉ")
print("="*60)
print(f"Temps d'entraînement: {training_time/60:.2f} minutes")
print(f"Heure de fin: {end_time.strftime('%H:%M:%S')}")

## 📈 5. Visualisation de l'Entraînement

In [ ]:
# Courbes d'entraînement
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Loss
axes[0, 0].plot(history.history['loss'], label='Train', linewidth=2)
axes[0, 0].plot(history.history['val_loss'], label='Validation', linewidth=2)
axes[0, 0].set_xlabel('Epoch', fontweight='bold')
axes[0, 0].set_ylabel('Loss', fontweight='bold')
axes[0, 0].set_title('Loss pendant l\'entraînement', fontweight='bold')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Accuracy
axes[0, 1].plot(history.history['accuracy'], label='Train', linewidth=2)
axes[0, 1].plot(history.history['val_accuracy'], label='Validation', linewidth=2)
axes[0, 1].set_xlabel('Epoch', fontweight='bold')
axes[0, 1].set_ylabel('Accuracy', fontweight='bold')
axes[0, 1].set_title('Accuracy pendant l\'entraînement', fontweight='bold')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Precision
axes[1, 0].plot(history.history['precision'], label='Train', linewidth=2)
axes[1, 0].plot(history.history['val_precision'], label='Validation', linewidth=2)
axes[1, 0].set_xlabel('Epoch', fontweight='bold')
axes[1, 0].set_ylabel('Precision', fontweight='bold')
axes[1, 0].set_title('Precision pendant l\'entraînement', fontweight='bold')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Recall
axes[1, 1].plot(history.history['recall'], label='Train', linewidth=2)
axes[1, 1].plot(history.history['val_recall'], label='Validation', linewidth=2)
axes[1, 1].set_xlabel('Epoch', fontweight='bold')
axes[1, 1].set_ylabel('Recall', fontweight='bold')
axes[1, 1].set_title('Recall pendant l\'entraînement', fontweight='bold')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('results/figures/lstm_training_history.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Graphiques sauvegardés!")

## 🎯 6. Évaluation sur le Test Set

In [ ]:
print("\n" + "="*60)
print("📊 ÉVALUATION SUR LE TEST SET")
print("="*60 + "\n")

# Prédictions
print("Génération des prédictions...")
y_pred_proba = lstm_model.predict(X_test, verbose=0)
y_pred = (y_pred_proba >= 0.5).astype(int)
print("✅ Prédictions générées\n")

# Calculer les métriques
results = {
    'model_name': 'LSTM',
    'precision_micro': precision_score(y_test, y_pred, average='micro', zero_division=0),
    'precision_macro': precision_score(y_test, y_pred, average='macro', zero_division=0),
    'recall_micro': recall_score(y_test, y_pred, average='micro', zero_division=0),
    'recall_macro': recall_score(y_test, y_pred, average='macro', zero_division=0),
    'f1_micro': f1_score(y_test, y_pred, average='micro', zero_division=0),
    'f1_macro': f1_score(y_test, y_pred, average='macro', zero_division=0),
    'hamming_loss': hamming_loss(y_test, y_pred),
    'subset_accuracy': accuracy_score(y_test, y_pred),
    'training_time_minutes': training_time / 60,
    'num_parameters': lstm_model.count_params()
}

# Afficher les résultats
print("📈 RÉSULTATS:")
print("-" * 60)
for metric, value in results.items():
    if metric not in ['model_name', 'training_time_minutes', 'num_parameters']:
        print(f"{metric:25s}: {value:.4f}")
print("-" * 60)
print(f"Temps d'entraînement: {results['training_time_minutes']:.2f} minutes")
print(f"Paramètres du modèle: {results['num_parameters']:,}")

## 📊 7. Analyse Détaillée par Classe

In [ ]:
# Calcul des métriques par classe
precision_per_class = precision_score(y_test, y_pred, average=None, zero_division=0)
recall_per_class = recall_score(y_test, y_pred, average=None, zero_division=0)
f1_per_class = f1_score(y_test, y_pred, average=None, zero_division=0)

# Créer un DataFrame
per_class_df = pd.DataFrame({
    'Emotion': emotion_labels,
    'Precision': precision_per_class,
    'Recall': recall_per_class,
    'F1-Score': f1_per_class
})

# Trier par F1-Score
per_class_df = per_class_df.sort_values('F1-Score', ascending=False)

print("\n📊 Métriques par émotion (Top 10):")
print(per_class_df.head(10).to_string(index=False))

print("\n📊 Métriques par émotion (Bottom 5):")
print(per_class_df.tail(5).to_string(index=False))

In [ ]:
# Visualisation des métriques par classe
fig, ax = plt.subplots(figsize=(16, 8))

x = np.arange(len(emotion_labels))
width = 0.25

ax.bar(x - width, precision_per_class, width, label='Precision', alpha=0.8)
ax.bar(x, recall_per_class, width, label='Recall', alpha=0.8)
ax.bar(x + width, f1_per_class, width, label='F1-Score', alpha=0.8)

ax.set_xlabel('Émotion', fontweight='bold', fontsize=12)
ax.set_ylabel('Score', fontweight='bold', fontsize=12)
ax.set_title('Métriques par Émotion - Modèle LSTM', fontweight='bold', fontsize=14)
ax.set_xticks(x)
ax.set_xticklabels(emotion_labels, rotation=45, ha='right')
ax.legend()
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('results/figures/lstm_per_class_metrics.png', dpi=300, bbox_inches='tight')
plt.show()

## 📈 7. Courbes ROC par Classe

In [ ]:
# 📊 Tracé des courbes ROC pour les 10 meilleures classes (par AUC-ROC)
print("\n" + "="*60)
print("📈 GÉNÉRATION DES COURBES ROC")
print("="*60 + "\n")

# Sélectionner les 10 classes avec les meilleurs AUC-ROC
top_10_classes = per_class_df.head(10)['Emotion'].tolist()
top_10_indices = [emotion_labels.index(em) for em in top_10_classes]

# Créer le graphique
fig, ax = plt.subplots(figsize=(12, 8))

# Couleurs variées
colors = plt.cm.tab10(np.linspace(0, 1, 10))

for idx, (class_idx, color) in enumerate(zip(top_10_indices, colors)):
    # Calculer la courbe ROC pour cette classe
    fpr, tpr, _ = roc_curve(y_test[:, class_idx], y_pred_proba[:, class_idx])
    roc_auc = auc(fpr, tpr)
    
    # Tracer la courbe
    ax.plot(fpr, tpr, color=color, lw=2, 
            label=f'{emotion_labels[class_idx]} (AUC = {roc_auc:.3f})')

# Ligne diagonale (classificateur aléatoire)
ax.plot([0, 1], [0, 1], 'k--', lw=2, label='Hasard (AUC = 0.500)')

ax.set_xlim([0.0, 1.0])
ax.set_ylim([0.0, 1.05])
ax.set_xlabel('Taux de Faux Positifs', fontweight='bold', fontsize=12)
ax.set_ylabel('Taux de Vrais Positifs', fontweight='bold', fontsize=12)
ax.set_title('Courbes ROC - Top 10 Émotions (LSTM)', fontweight='bold', fontsize=14)
ax.legend(loc='lower right', fontsize=10)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('results/figures/lstm_roc_curves.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Courbes ROC générées et sauvegardées: results/figures/lstm_roc_curves.png")

In [ ]:
# 🎯 Calcul de l'AUC-ROC (requis par l'énoncé)
print("\n" + "="*60)
print("📈 CALCUL DE L'AUC-ROC")
print("="*60 + "\n")

# AUC-ROC Micro (moyenne globale sur toutes les classes)
auc_micro = roc_auc_score(y_test, y_pred_proba, average='micro')
print(f"AUC-ROC (Micro): {auc_micro:.4f}")

# AUC-ROC Macro (moyenne non pondérée par classe)
auc_macro = roc_auc_score(y_test, y_pred_proba, average='macro')
print(f"AUC-ROC (Macro): {auc_macro:.4f}")

# AUC-ROC par classe
auc_per_class = roc_auc_score(y_test, y_pred_proba, average=None)
per_class_df['AUC-ROC'] = auc_per_class
per_class_df = per_class_df.sort_values('AUC-ROC', ascending=False)

print("\n📊 AUC-ROC par émotion (Top 10):")
print(per_class_df[['Emotion', 'AUC-ROC']].head(10).to_string(index=False))

print("\n📊 AUC-ROC par émotion (Bottom 5):")
print(per_class_df[['Emotion', 'AUC-ROC']].tail(5).to_string(index=False))

## 💾 8. Sauvegarde des Résultats

In [ ]:
print("Sauvegarde des résultats...\n")

# 💾 Sauvegarder le modèle au format .pickle (requis par l'énoncé)
print("📦 Sauvegarde du modèle au format .pickle...")
model_data = {
    'model_config': lstm_model.get_config(),
    'model_weights': lstm_model.get_weights(),
    'vocab_size': vocab_size,
    'max_length': max_length,
    'embedding_dim': embedding_dim,
    'emotion_labels': emotion_labels
}
with open('models/lstm/lstm_model.pickle', 'wb') as f:
    pickle.dump(model_data, f)
print("✅ Modèle sauvegardé: models/lstm/lstm_model.pickle")

# Sauvegarder aussi au format .h5 pour compatibilité
lstm_model.save('models/lstm/final_model.h5')
print("✅ Modèle sauvegardé (format .h5): models/lstm/final_model.h5")

# Sauvegarder l'historique d'entraînement
with open('models/lstm/training_history.pkl', 'wb') as f:
    pickle.dump(history.history, f)
print("✅ Historique sauvegardé: models/lstm/training_history.pkl")

# Mettre à jour les résultats avec AUC-ROC
results['auc_roc_micro'] = float(auc_micro)
results['auc_roc_macro'] = float(auc_macro)

# Sauvegarder les résultats au format JSON
with open('results/metrics/lstm_results.json', 'w') as f:
    json.dump(results, f, indent=4)
print("✅ Résultats sauvegardés: results/metrics/lstm_results.json")

# Sauvegarder les prédictions
np.save('models/lstm/test_predictions.npy', y_pred)
np.save('models/lstm/test_probabilities.npy', y_pred_proba)
print("✅ Prédictions sauvegardées")

# Sauvegarder les métriques par classe (avec AUC-ROC)
per_class_df.to_csv('results/metrics/lstm_per_class_metrics.csv', index=False)
print("✅ Métriques par classe sauvegardées: results/metrics/lstm_per_class_metrics.csv")

print("\n" + "="*60)
print("🎉 LSTM - ENTRAÎNEMENT ET ÉVALUATION TERMINÉS !")
print("="*60)
print("\nProcédez au notebook suivant:")
print("  📓 Personne_2_BiLSTM_Attention.ipynb")

## 📤 9. [Optionnel] Sauvegarder vers Google Drive

In [ ]:
# Monter Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Copier les résultats
!mkdir -p "/content/drive/MyDrive/emotion_detection_project"
!cp -r models/lstm "/content/drive/MyDrive/emotion_detection_project/"
!cp -r results "/content/drive/MyDrive/emotion_detection_project/"

print("\n✅ Résultats LSTM sauvegardés dans Google Drive!")